# Data Engineer Certification Sample Practical Exam

HappyPaws, creates fun and educational apps for pet owners. 

HappyPaws wants to help pet owners understand their pets better by tracking their activities and health through the app. 

The data engineering team is responsible for making sure all the pet data from thousands of users is organized and safe, so pet owners can get tips to keep their pets happy and healthy.



## Task

HappyPaws has collected three datasets over the past year: 
 - "pet_activities.csv" which logs daily activities of pets, 
 - "pet_health.csv" which records vet visits and health issues, and 
 - "users.csv" which contains information about the pet owners. 

Each dataset contains unique identifiers for pets and/or their owners. 

The engineers developing the app currently write code to cross reference all of these data sources. 

They want to make things easier by having a single table with all data included.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset. 

The final dataset should provide a comprehensive view of each pet's activities, health records, and owner information. 

- To test your code, your manager will run only the code `all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv')`
- Your `all_pet_data()` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.

## Data

The data that has been provided has the following structure

![image](image.png)

The function that you write must return data as described below. There should be a unique row for each activity/health visit. 

Where missing values are permitted, they should be in the default Python format.

|Column Name | Description |
|-------|-------|
| pet_id | Unique identifier for each pet. There should not be any missing values.|
| date | The date of the activity recorded or the date of the health visit, in date format. There should not be any missing values.|
| activity_type | The type of activity, one of 'Walking', 'Playing', 'Resting' or for rows that relate to a health visit, the value 'Health'. Missing values are permitted.|
| duration_minutes | The duration of the activity in minutes. For rows that relate to health visits, this should be 0. Missing values for other activities are permitted.|
| issue | The health issue identified or check-up note. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| resolution | The outcome or advice given for the issue. For rows that relate to activities, this should be a missing value. Missing values for health activities are permitted.|
| owner_id | Unique identifier for the pet owner. All pets must have an owner.|
| owner_age_group | The age group of the owner (e.g., 18-25, 26-35, etc.). Missing values are permitted.|
| pet_type | The type of pet (e.g., Dog, Cat). Missing values are permitted.|

In [324]:
# Imports
import pandas as pd
import numpy as np

In [325]:
# Functions

def load_dbs(*filenames: str):
    """Takes supplied filenames and returns a list of dataframes \
    
    The list order is 'activities', 'health', 'users'

    Returns:
        list: a list of dataframes
    """
    names = ['activities', 'health', 'users']
    dataframes = []
    for name in names:
        result = list(filter(lambda filename: name in filename, filenames))
        if not result:
            raise ValueError(f'None of the supplied filenames contains "{name}"')
        if len(result) > 1:
            raise ValueError(f'More than one filename returned containing "{name}"')
        dataframes.append(pd.read_csv(result[0]))
    return dataframes

def merge_dbs(*dfs: pd.DataFrame):
    """Merges the supplied databases \
    
    Expects databases to be two or more of "pet activities", "pet health", and "users"

    Returns:
        pandas.DataFrame: the resulting merged datafram
    """
    # Identify dataframes
    activities = list(filter(lambda x: 'activity_type' in x.columns, dfs))
    health = list(filter(lambda x: 'issue' in x.columns, dfs))
    users = list(filter(lambda x: 'owner_id' in x.columns, dfs))

    # Check for at least two valid databases
    if sum([bool(x) for x in [activities, health, users]]) < 2:
        raise ValueError('Must supply at least two valid dataframes')

    # Rename columns to simplify merge
    if health:
        health[0].rename(columns={'visit_date': 'date'}, inplace=True)

    # Merge dataframes
    if activities and users:
        activities = pd.merge(activities[0], users[0], on='pet_id')
    else:
        activities = activities[0]
    if health and users:
        health = pd.merge(health[0], users[0], on='pet_id')
    else:
        health = health[0]
    merged_df = pd.concat([activities, health])
    sorted_merged_df = merged_df.sort_values(['date'])
    return sorted_merged_df

def clean_data(df: pd.DataFrame):
    """Checks for specific columns in the dataframe and cleans them

    Args:
        df (pandas.DataFrame): the dataframe to be cleaned

    Returns:
        pandas.DataFrame: the cleaned dataframe
    """
    tdf = df.copy()
    # Change "date" datatype
    if 'date' in tdf.columns:
        tdf.date = pd.to_datetime(tdf.date, errors='coerce')

    # Assign "Health" to health activities
    if 'issue' in tdf.columns:
        tdf.loc[(~df.issue.isna()) & (tdf.activity_type.isna()), 'activity_type'] = 'Health'  

    # Clean values for 'activity_type': "Playing", "Walking", "Resting"
    if 'activity_type' in tdf.columns:
        for activity in ['Play', 'Walk', 'Rest']:
            tdf.activity_type = tdf.activity_type.str.replace(fr'^{activity}$', f'{activity}ing', regex=True)
        tdf.activity_type = tdf.activity_type.astype('category')  

    # Assign 0 to "duration_minutes" for health records and convert datatype
    if 'duration_minutes' in tdf.columns:
        tdf.loc[tdf.activity_type == 'Health', 'duration_minutes'] = 0
        tdf.loc[tdf.duration_minutes == '-', 'duration_minutes'] = np.nan
        tdf.duration_minutes = tdf.duration_minutes.astype('float')

    # Assign None for "issue", if missing, and change datatype to "category"
    if 'issue' in tdf.columns:
        tdf.loc[tdf.issue.isna(), 'issue'] = np.nan
        tdf.issue = tdf.issue.astype('category')

    # Assign None for "resolution", if missing, and change datatype to "category"
    if 'resolution' in tdf.columns:
        tdf.loc[tdf.resolution.isna(), 'resolution'] = np.nan
        tdf.resolution = tdf.resolution.astype('category')

    # Change owner age group to datatype to "category" and assign NaN, if missing
    if 'owner_age_group' in tdf.columns:
        tdf.loc[tdf.owner_age_group.isna(), 'owner_age_group'] = np.nan
        tdf.owner_age_group = tdf.owner_age_group.astype('category')

    return tdf

def all_pet_data(activities: str, health: str, users: str):
    """Merges and cleans data from the "pet activities", "pet health", and "users" databases

    Args:
        activities (str): the "activities" CSV file
        health (str): the "health" CSV file
        users (str): the "users" CSV file

    Returns:
        pandas.DataFrame: a DataFrame of cleaned and merged databases
    """
    df_list = load_dbs(activities, health, users)  # Load
    merged = merge_dbs(*df_list)
    cleaned = clean_data(merged)
    col_order = ['pet_id', 'date', 'activity_type', 'duration_minutes', 'issue', 'resolution',	'owner_id',	'owner_age_group', 'pet_type']
    sorted_df = cleaned.loc[:, col_order]
    return sorted_df

In [326]:
df = all_pet_data('pet_activities.csv', 'pet_health.csv', 'users.csv')
df.resolution.value_counts(dropna=False)

resolution
NaN                                                     938
Cleaning done, no dental issues                         200
Diet adjustment recommended for weight management       163
All clear, healthy diet recommended                     159
Vaccination completed, next due in 1 year               157
Medication and follow-up in 2 weeks for infection        86
Sprain identified, rest and limited movement advised     67
Minor surgery performed, recovery monitored              60
Physical therapy recommended for joint issues            48
Name: count, dtype: int64